установка необходимых библиотек

In [20]:
! pip install datasets docx2txt langchain langchain_community

In [30]:
import openai
import numpy as np
from typing import List
from pathlib import Path
import fitz
import re
from docx import Document

from tqdm.auto import tqdm
import pandas as pd

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Инициализация клиента и моделей

In [2]:
client = openai.OpenAI(
    api_key="sk-TeK03KzPC8jTYJP7Pmf60w",
    base_url="https://api.vsellm.ru/v1"
)

embed_model_name = 'openai/text-embedding-3-small'
generative_model_name = "openai/gpt-4.1-mini"

## Пример использования генеративных моделей

Генерация эмбедингов

In [3]:
texts = [
    "Graph Neural Networks are powerful",
    "GNNs are used in recommender systems",
    "Transformers work well for NLP"
]

response = client.embeddings.create(
    model=embed_model_name,
    input=texts
)

embeddings = [item.embedding for item in response.data]

print(len(embeddings))          # количество текстов
print(len(embeddings[0]))       # размерность эмбеддинга (3072)

3
1536


Генерация текста

In [4]:
response = client.chat.completions.create(
    model=generative_model_name,
    messages=[
        {"role": "user", "content": "Привет!"}
    ]
)

print(response.choices[0].message.content)

Привет! Как могу помочь?


## Baseline HACK2025


## Загрузка базы знаний

In [13]:
def load_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return re.sub(r'\s+', ' ', text).strip()

def load_pdf(file_path):
    text = ""
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text("text") + "\n"
    return re.sub(r'\s+', ' ', text).strip()

data_folder = Path("lectures") 

texts = []
for file in data_folder.glob("*"):
    if file.suffix == ".docx":
        print(f"Загрузка DOCX: {file.name}")
        texts.append(load_docx(file))
    elif file.suffix == ".pdf":
        print(f"Загрузка PDF: {file.name}")
        texts.append(load_pdf(file))
        
print(f"\nНайдено файлов: {len(texts)}")

full_text = "\n".join(texts)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_text(full_text)

print(f"Total chunks: {len(chunks)}")

Загрузка PDF: GNN_module1.pdf
Загрузка PDF: GNN_module2.pdf
Загрузка PDF: module3.pdf
Загрузка PDF: module4.pdf
Загрузка PDF: module5.pdf
Загрузка PDF: module6.pdf
Загрузка PDF: module7.pdf
Загрузка PDF: module8.pdf
Загрузка DOCX: Курс.docx

Найдено файлов: 9
Total chunks: 380


## Генерация эмбеддингов для чанков

In [17]:
# обновлено для работы не только с docx
def embed_documents(texts):
    embeddings = []
    for i in tqdm(range(0, len(texts), 50)):  # батчами по 50
        batch = texts[i:i+50]
        response = client.embeddings.create(
            model=embed_model_name,
            input=batch
        )
        embeddings.extend([item.embedding for item in response.data])
    return np.array(embeddings)

chunk_embeddings = embed_documents(chunks)
print("Эмбеддинги сгенерированы:", chunk_embeddings.shape)
print("Пример:", chunk_embeddings[0][:5])

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:17<00:00,  9.66s/it]

Эмбеддинги сгенерированы: (380, 1536)
Пример: [-0.04010519 -0.04786523  0.00559865 -0.00976413  0.04092081]


## Retrieval (cosine similarity)

In [25]:
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def retrieve(query, k=5):
    query_emb = client.embeddings.create(
        model=embed_model_name,
        input=[query]
    ).data[0].embedding

    scores = [
        cosine_sim(query_emb, emb)
        for emb in chunk_embeddings
    ]

    top_idx = np.argsort(scores)[-k:][::-1]

    return [
        {
            "text": chunks[i],
            "score": float(scores[i])
        }
        for i in top_idx
    ]

## Сбор контекста

In [26]:
def build_context(retrieved_chunks):
    return "\n\n".join(
        f"[score={c['score']:.3f}]\n{c['text']}"
        for c in retrieved_chunks
    )

## Генерация ответа

In [20]:
def generate_answer(question, context):
    response = client.chat.completions.create(
        model=generative_model_name,
        messages=[
            {
                "role": "system",
                "content": (
                    "Ты ассистент, отвечающий на вопросы по курсу Graph Neural Networks. "
                    "Используй только предоставленный контекст."
                )
            },
            {
                "role": "user",
                "content": f"""
Контекст:
{context}

Вопрос:
{question}

Дай краткий и точный ответ.
"""
            }
        ],
        temperature=0.2
    )

    return response.choices[0].message.content

## Финальная функция answer()

In [27]:
def answer(question, k=5):
    retrieved = retrieve(question, k)
    context = build_context(retrieved)
    answer_text = generate_answer(question, context)

    return {
        "question": question,
        "answer": answer_text,
        "retrieved_chunks": retrieved
    }

In [28]:
result = answer("Что такое Graph Neural Network?")

print(result["answer"])

Graph Neural Network (GNN) — это класс нейронных сетей для обработки данных, которые можно представить в виде графов.


In [32]:
qa_df = pd.read_excel("QA_dataset.xlsx")
questions = qa_df["Question"].dropna().tolist()
answers = []

for q in tqdm(questions):
    res = answer(q, k=5)
    answers.append(res["answer"])

qa_df["Answer"] = answers
qa_df.to_excel("QA_dataset_answered_baseline.xlsx", index=False)

qa_df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:30<00:00,  1.50s/it]


,Question,Answer
0,Что такое граф и как он формально задаётся?,"Граф — это структура данных, описывающая объек..."
1,Какие основные компоненты используются для опи...,Основные компоненты динамического графа: множе...
2,В чём отличие snapshot-based и event-based дин...,Snapshot-based динамика представляет граф как ...
3,Какие задачи решают динамические GNN?,Динамические GNN решают задачи: 1) временное п...
4,Почему для динамических графов требуется памят...,Для динамических графов требуется память узлов...


#### Где можно вдохновиться: RAG From Scratch